In [1]:
import numpy as np
import pandas as pd

from ols_bootstrap.pairs import PairsBootstrap
from ols_bootstrap.residual import ResidualBootstrap
from ols_bootstrap.wild import WildBootstrap

In [2]:
df = pd.read_csv('./balance2018.csv')
df = df[df['sales_clean'] != 0]
df = df[['sales_clean', 'tanass_clean', 'tax']]
df = df.dropna(subset=['tanass_clean', 'tax'])

df_scaled = df.applymap(lambda x: np.log(x + 1))

/home/phamv/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (66) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
df_sample = df_scaled.sample(n=10000, replace=False)

Y_data = pd.DataFrame(df_sample.iloc[:, 0])
X_data = pd.DataFrame(df_sample.iloc[:, 1:])

## Default SE on the original OLS is HC3, default CI on bootstrapped parameter is BC. 

In [4]:
psb = PairsBootstrap(Y_data, X_data, reps = 1000)
psb.fit()

In [5]:
rsb = ResidualBootstrap(Y_data, X_data, reps = 1000)
rsb.fit()

In [6]:
wb_stdn = WildBootstrap(Y_data, X_data, reps = 1000, from_distro = "standard_normal")
wb_stdn.fit()

In [7]:
wb_rad = WildBootstrap(Y_data, X_data, reps = 1000, from_distro = "rademacher")
wb_rad.fit()

In [8]:
wb_mam = WildBootstrap(Y_data, X_data, reps = 1000, from_distro = "mammen")
wb_mam.fit()

In [9]:
psb.summary()

+----------------------------------------------------------------------------------------------------------------------------------+
|                     Pairs Bootstrap results with 10000 obs and 1000 BS reps using HC3 SE-s and 95.00% BC CI                      |
+--------------+------------+------------------+--------+---------------+--------------+--------------+------------------+---------+
|     Var      | OLS Params | Avg of BS Params |  Bias  | OLS Params SE | BS Params SE | % of SE Diff |        CI        | CI Diff |
+--------------+------------+------------------+--------+---------------+--------------+--------------+------------------+---------+
|    const     |   6.1778   |      6.1775      | 0.0002 |     0.0440    |    0.0446    |    -1.39     | [6.0923, 6.2645] |  0.1722 |
+--------------+------------+------------------+--------+---------------+--------------+--------------+------------------+---------+
| tanass_clean |   0.1788   |      0.1788      | 0.0000 |     0.0051 

In [10]:
rsb.summary()

+----------------------------------------------------------------------------------------------------------------------------------+
|                    Residual Bootstrap results with 10000 obs and 1000 BS reps using HC3 SE-s and 95.00% BC CI                    |
+--------------+------------+------------------+--------+---------------+--------------+--------------+------------------+---------+
|     Var      | OLS Params | Avg of BS Params |  Bias  | OLS Params SE | BS Params SE | % of SE Diff |        CI        | CI Diff |
+--------------+------------+------------------+--------+---------------+--------------+--------------+------------------+---------+
|    const     |   6.1778   |      6.1783      | 0.0005 |     0.0440    |    0.0355    |    19.29     | [6.1044, 6.2455] |  0.1410 |
+--------------+------------+------------------+--------+---------------+--------------+--------------+------------------+---------+
| tanass_clean |   0.1788   |      0.1787      | 0.0000 |     0.0051 

In [11]:
wb_stdn.summary()

+----------------------------------------------------------------------------------------------------------------------------------+
|           Wild Bootstrap with Standard Normal results with 10000 obs and 1000 BS reps using HC3 SE-s and 95.00% BC CI            |
+--------------+------------+------------------+--------+---------------+--------------+--------------+------------------+---------+
|     Var      | OLS Params | Avg of BS Params |  Bias  | OLS Params SE | BS Params SE | % of SE Diff |        CI        | CI Diff |
+--------------+------------+------------------+--------+---------------+--------------+--------------+------------------+---------+
|    const     |   6.1778   |      6.1783      | 0.0005 |     0.0440    |    0.0351    |    20.18     | [6.1107, 6.2471] |  0.1365 |
+--------------+------------+------------------+--------+---------------+--------------+--------------+------------------+---------+
| tanass_clean |   0.1788   |      0.1787      | 0.0001 |     0.0051 

In [12]:
wb_rad.summary()

+----------------------------------------------------------------------------------------------------------------------------------+
|              Wild Bootstrap with Rademacher results with 10000 obs and 1000 BS reps using HC3 SE-s and 95.00% BC CI              |
+--------------+------------+------------------+--------+---------------+--------------+--------------+------------------+---------+
|     Var      | OLS Params | Avg of BS Params |  Bias  | OLS Params SE | BS Params SE | % of SE Diff |        CI        | CI Diff |
+--------------+------------+------------------+--------+---------------+--------------+--------------+------------------+---------+
|    const     |   6.1778   |      6.1777      | 0.0000 |     0.0440    |    0.0352    |    19.89     | [6.1098, 6.2489] |  0.1391 |
+--------------+------------+------------------+--------+---------------+--------------+--------------+------------------+---------+
| tanass_clean |   0.1788   |      0.1788      | 0.0000 |     0.0051 

In [13]:
wb_mam.summary()

+----------------------------------------------------------------------------------------------------------------------------------+
|                Wild Bootstrap with Mammen results with 10000 obs and 1000 BS reps using HC3 SE-s and 95.00% BC CI                |
+--------------+------------+------------------+--------+---------------+--------------+--------------+------------------+---------+
|     Var      | OLS Params | Avg of BS Params |  Bias  | OLS Params SE | BS Params SE | % of SE Diff |        CI        | CI Diff |
+--------------+------------+------------------+--------+---------------+--------------+--------------+------------------+---------+
|    const     |   6.1778   |      6.1761      | 0.0017 |     0.0440    |    0.0355    |    19.36     | [6.1026, 6.2431] |  0.1404 |
+--------------+------------+------------------+--------+---------------+--------------+--------------+------------------+---------+
| tanass_clean |   0.1788   |      0.1790      | 0.0002 |     0.0051 

## Some useful methods were implemented (let's use it on wb_mam object of Wild Bootstrap with Mammen)

### get_ci() method

#### Vanila version is when only the actual CI was used with all independent variables

In [14]:
wb_mam.get_ci()

bc        
                 lwb     upb
const         6.1026  6.2431
tanass_clean  0.1712  0.1866
tax           0.5062  0.5318

#### However, any combination of ('bc', 'bca', 'percentile) CI types could be selected and any combinations of independent variables can be chosen with 'which_ci' and 'which_var' optional arguments, respectively.

Please note that if choosing 'bca' the calculation can take a while as it uses jacknife resampling for calculating the acceleration factor

In [15]:
wb_mam.get_ci(which_ci=['bc', 'percentile'], which_var=['tax', 'const'])

bc         percentile        
          lwb     upb        lwb     upb
tax    0.5062  0.5318     0.5058  0.5316
const  6.1026  6.2431     6.1031  6.2441

### get_all_se() method

The following SE-s are calculated: 
- bootstrapped - standard error of the bootstrapped parameters
- constant - model-based ols standard error, that is, constant variance is assumed 
- hc0, hc1, hc2, hc3, hc4, hc4m, hc5 - Heteroscedastic-consistan standard errors using sandwich estimators 

#### Vanila version is when using all indepencdent variables. 

In [16]:
wb_mam.get_all_se()

,bootstrapped,constant,hc0,hc1,hc2,hc3,hc4,hc4m,hc5
const,0.0355,0.0351,0.0440,0.0440,0.0440,0.0440,0.0440,0.0440,0.0440
tanass_clean,0.0039,0.0039,0.0051,0.0051,0.0051,0.0051,0.0051,0.0051,0.0051
tax,0.0066,0.0066,0.0093,0.0093,0.0093,0.0093,0.0093,0.0093,0.0093


#### Can provide a subset of indepent vars to calculate the above-mentioned 9 SE-s with 'which_var' argument

In [17]:
wb_mam.get_all_se(which_var=['tanass_clean', 'tax'])

,bootstrapped,constant,hc0,hc1,hc2,hc3,hc4,hc4m,hc5
tanass_clean,0.0039,0.0039,0.0051,0.0051,0.0051,0.0051,0.0051,0.0051,0.0051
tax,0.0066,0.0066,0.0093,0.0093,0.0093,0.0093,0.0093,0.0093,0.0093
